# Kuala Lumpur Neighborhood Classification

### Summary
* [Introduction & General Information](http://localhost:8888/notebooks/Kuala%20Lumpur%20Neighborhood%20Classification.ipynb#Introduction-&-General-Information)
* [Objectives](http://localhost:8888/notebooks/Kuala%20Lumpur%20Neighborhood%20Classification.ipynb#Objectives)
* [Data Used](http://localhost:8888/notebooks/Kuala%20Lumpur%20Neighborhood%20Classification.ipynb#Data-Used)
* [Approach & Methodology](http://localhost:8888/notebooks/Kuala%20Lumpur%20Neighborhood%20Classification.ipynb#Approach-&-Methodology)
* [Observations](http://localhost:8888/notebooks/Kuala%20Lumpur%20Neighborhood%20Classification.ipynb#Observations:)
* [Recommendation](http://localhost:8888/notebooks/Kuala%20Lumpur%20Neighborhood%20Classification.ipynb#Recommendations:)

### Introduction & General Information
(source: https://www.wikiwand.com/en/Kuala_Lumpur)

* Kuala Lumpur, officially the Federal Territory of Kuala Lumpur and colloquially referred to as KL, is a federal territory and the capital city of Malaysia. 
* It is the largest city in Malaysia, covering an area of 243 sq km (94 sq mi) with an estimated population of 1.73 million as of 2016.
* Greater Kuala Lumpur, also known as the Klang Valley, is an urban agglomeration of 7.564 million people as of 2018.
* It is among the fastest growing metropolitan regions in Southeast Asia, in both population and economic development.

### Objectives

* Selection of a prime location while opening a new restaurant business is one of the most important decision, having a profound impact on the business's success or failure.
  * If the restaurant is opened in an area where there is already a very high competition and established businesses of the same kind, the new restaurant might not be as successful. On the other hand, if it is opened in an area where there is very less competition, the new business might get established in very short period and start earning good profits.

* This project is aimed at analyzing the data and using different machine learning techniques to determine the best possible cluster of neighborhoods where a new restaurant will have the highest possible chance at success.

### Data Used

* In this project, the neighborhood data is collected from Wikipedia pages by scrapping and post-processing.
* In addition to that, [Foursquare API](https://developer.foursquare.com/docs/) is used to explore different venues within the neighborhoods, based on different restaurant categories. 

### Approach & Methodology

* Perform web scrapping and post-processing of wikipedia pages to obtain information on different neighborhoods of Kuala Lumpur.
* Get latitude and longitude of above neighborhoods using [Geocoder Package](https://geocoder.readthedocs.io/).
* Use [Foursquare API](https://developer.foursquare.com/docs/) to get data of different venues within the neighborhoods.
* Group the venue data by neighborhood and calculate the mean frequency of each restaurant category within the venues.
* Filter the resulting venue categories by restaurants.
* Apply K-means Clustering to generate different clusters from above data.
* Visualize the generated clusters in a map of Kuala Lumpur using [Folium](https://python-visualization.github.io/folium/).
* Analyze the results.

### 1. Importing Libraries

In [17]:
import numpy as np

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 
import geocoder
print("Libraries imported.")

Libraries imported.


### 2. Scrapping data from Wikipedia page into a DataFrame

In [18]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [19]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [20]:
# create a list to store neighborhood data
neighborhood_List = []

In [21]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhood_List.append(row.text)

In [22]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhood_List})

kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [23]:
# print the number of rows of the dataframe
kl_df.shape

(71, 1)

### 3. Getting the geographical coordinates

In [24]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [25]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [26]:
coords[:10]

[[3.0576900000000364, 101.74388000000005],
 [3.1484988508598852, 101.69672774991264],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.0827600000000643, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008]]

In [27]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [28]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [29]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df.head()

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148499,101.696728
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


### 4. Creating a map of Kuala Lumpur with neighborhoods superimposed on top

In [30]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="neighborhood-classification")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516964, 101.6942371.


In [32]:
# create map of Kuala Lumpur using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

### 5. Using the Foursquare API to explore the neighborhoods

In [33]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [34]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
     # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [35]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7082, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
1,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,Darma Motor,3.054160,101.744328,Motorcycle Shop
4,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant


**Let's check how many venues were returned for each neighorhood**

In [36]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,99,99,99,99,99,99
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [37]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 308 uniques categories.


In [38]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Food & Drink Shop', 'Supplement Shop', 'Noodle House',
       'Motorcycle Shop', 'Chinese Restaurant', 'Seafood Restaurant',
       'Breakfast Spot', 'Restaurant', 'Vegetarian / Vegan Restaurant',
       'Food Court', 'Other Great Outdoors', 'Asian Restaurant',
       'Indian Restaurant', 'Park', 'Bubble Tea Shop', 'Coffee Shop',
       'Dim Sum Restaurant', 'Convenience Store', 'Spa',
       'Japanese Restaurant', 'Snack Place', 'Dessert Shop',
       'Outlet Store', 'Chinese Breakfast Place', 'Café', 'Pet Store',
       'Bakery', 'Farmers Market', 'Gym / Fitness Center',
       'Malay Restaurant', 'Mamak Restaurant',
       'Middle Eastern Restaurant', 'Fast Food Restaurant',
       'Cantonese Restaurant', 'Hakka Restaurant', 'Athletics & Sports',
       'Steakhouse', 'College Bookstore', 'Burger Joint', 'Winery',
       'Food Truck', 'Badminton Court', 'Halal Restaurant',
       'Grocery Store', 'Korean Restaurant', 'Salon / Barbershop',
       'Hostel', 'South Indian Restau

### 6. Analyzing Each Neighborhood

In [39]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7082, 309)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Betting Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Pastry Shop,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex 

In [40]:
kl_onehot_filtered_restaurants = kl_onehot.filter(regex='Restaurant$',axis=1)
k1_onehot_neighborhoods = kl_onehot['Neighborhoods']

kl_onehot = pd.concat([k1_onehot_neighborhoods, kl_onehot_filtered_restaurants], axis=1)

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [42]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped.head()

(71, 62)


,Neighborhoods,African Restaurant,American Restaurant,Asian Restaurant,Brazilian Restaurant,Cantonese Restaurant,Chettinad Restaurant,Chinese Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Fast Food Restaurant,French Restaurant,Grilled Meat Restaurant,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hawaiian Restaurant,Hong Kong Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Indonesian Restaurant,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Latin American Restaurant,Malay Restaurant,Mamak Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,North Indian Restaurant,Pakistani Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Satay Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,South American Restaurant,South Indian Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yakitori Restaurant
0,Alam Damai,0.00,0.0,0.07000,0.0,0.01,0.00,0.190000,0.000000,0.020000,0.000000,0.0,0.010000,0.0,0.0,0.0,0.01,0.010000,0.00,0.0,0.00,0.0,0.030000,0.000000,0.0,0.00,0.01,0.00,0.01,0.0,0.00,0.100000,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.030000,0.0,0.0,0.02,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.00,0.01,0.0,0.00
1,"Ampang, Kuala Lumpur",0.00,0.0,0.02000,0.0,0.00,0.01,0.020000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.030000,0.000000,0.0,0.01,0.01,0.01,0.00,0.0,0.02,0.040000,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.020000,0.0,0.0,0.00,0.0,0.0,0.02,0.01,0.0,0.01,0.0,0.0,0.01,0.0,0.000000,0.00,0.01,0.01,0.0,0.01
2,Bandar Menjalara,0.01,0.0,0.03000,0.0,0.00,0.00,0.080000,0.000000,0.000000,0.000000,0.0,0.010000,0.0,0.0,0.0,0.00,0.000000,0.01,0.0,0.01,0.0,0.010000,0.000000,0.0,0.00,0.03,0.00,0.01,0.0,0.00,0.000000,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.01,0.020000,0.0,0.0,0.00,0.0,0.0,0.00,0.01,0.0,0.00,0.0,0.0,0.00,0.0,0.030000,0.01,0.00,0.04,0.0,0.00
3,Bandar Sri Permaisuri,0.00,0.0,0.06000,0.0,0.00,0.00,0.130000,0.000000,0.000000,0.000000,0.0,0.030000,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.020000,0.030000,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.080000,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.030000,0.0,0.0,0.03,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.010000,0.00,0.00,0.00,0.0,0.00
4,Bandar Tasik Selatan,0.00,0.0,0.10101,0.0,0.00,0.00,0.060606,0.010101,0.010101,0.010101,0.0,0.010101,0.0,0.0,0.0,0.00,0.010101,0.00,0.0,0.00,0.0,0.040404,0.010101,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.090909,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.020202,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.040404,0.00,0.00,0.00,0.0,0.00


In [43]:
neighborhoods = kl_grouped['Neighborhoods']
restaurants = kl_grouped.filter(regex='Restaurant$',axis=1)

kl_restaurant = pd.concat([neighborhoods, restaurants.sum(axis=1)], axis=1)
kl_restaurant.rename(columns = {0:'Restaurant'}, inplace = True) 
kl_restaurant.head()

,Neighborhoods,Restaurant
0,Alam Damai,0.550000
1,"Ampang, Kuala Lumpur",0.290000
2,Bandar Menjalara,0.320000
3,Bandar Sri Permaisuri,0.420000
4,Bandar Tasik Selatan,0.414141


### 7. Clustering Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [44]:
# set number of clusters
kclusters = 3

kl_clustering = kl_restaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 2, 2, 0, 1, 1, 1, 0], dtype=int32)

In [45]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_restaurant.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [46]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Alam Damai,0.550000,0
1,"Ampang, Kuala Lumpur",0.290000,1
2,Bandar Menjalara,0.320000,2
3,Bandar Sri Permaisuri,0.420000,2
4,Bandar Tasik Selatan,0.414141,2


In [47]:
# merge kl_grouped with kl_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(71, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.550000,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.290000,1,3.148499,101.696728
2,Bandar Menjalara,0.320000,2,3.190350,101.625450
3,Bandar Sri Permaisuri,0.420000,2,3.103910,101.712260
4,Bandar Tasik Selatan,0.414141,2,3.072750,101.714610


In [48]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged.head()

(71, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.55,0,3.057690,101.743880
33,Kepong,0.50,0,3.217500,101.637630
34,Kepong Baru,0.50,0,3.207771,101.645173
69,Titiwangsa,0.48,0,3.180730,101.703210
40,Mont Kiara,0.43,0,3.165290,101.652420


**Finally, let's visualize the resulting clusters**

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 8. Examining Clusters

#### Cluster 0

In [50]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.550000,0,3.057690,101.743880
33,Kepong,0.500000,0,3.217500,101.637630
34,Kepong Baru,0.500000,0,3.207771,101.645173
69,Titiwangsa,0.480000,0,3.180730,101.703210
40,Mont Kiara,0.430000,0,3.165290,101.652420
41,Pantai Dalam,0.480000,0,3.094760,101.667470
43,Putrajaya,0.430000,0,3.125860,101.718624
44,Salak South,0.510000,0,3.081540,101.696890
46,Semarak,0.500000,0,3.179943,101.721449
48,Setapak,0.480000,0,3.188160,101.704150


#### Cluster 1

In [51]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
11,Brickfields,0.23,1,3.129160,101.684060
12,Bukit Bintang,0.29,1,3.147770,101.708550
8,Bangsar South,0.24,1,3.111020,101.662830
7,Bangsar Park,0.19,1,3.129200,101.678440
13,Bukit Jalil,0.30,1,3.057810,101.689650
24,"Federal Hill, Kuala Lumpur",0.23,1,3.136370,101.685640
6,Bangsar,0.19,1,3.129200,101.678440
15,Bukit Nanas,0.26,1,3.148609,101.699854
16,Bukit Petaling,0.28,1,3.129290,101.698960
57,Taman Duta,0.30,1,3.155620,101.671840


#### Cluster 2

In [52]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
3,Bandar Sri Permaisuri,0.420000,2,3.103910,101.712260
4,Bandar Tasik Selatan,0.414141,2,3.072750,101.714610
2,Bandar Menjalara,0.320000,2,3.190350,101.625450
25,Happy Garden,0.340000,2,3.201630,101.721070
47,"Sentul, Kuala Lumpur",0.390000,2,3.175080,101.693050
54,Taman Bukit Maluri,0.360000,2,3.200660,101.633370
53,Sungai Besi,0.390000,2,3.049970,101.706030
50,Shamelin,0.390000,2,3.124570,101.735970
49,Setiawangsa,0.400000,2,3.191802,101.740066
14,Bukit Kiara,0.330000,2,3.143480,101.644330


### Observations:

- Most of the restaurants are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 (shown in mint green color) and moderate number in cluster 1 (shown in red color). 
  - On the other hand, cluster 0 (shown in purple color) has comparatively very less number of restaurants in the neighborhoods. 
  - This represents a great opportunity and high potential areas to open new restaurant as there is very little competition from the existing ones. 
  - Meanwhile, restaurants in cluster 2 are most likely suffering from intense competition due to high concentration of restaurant options available. 
  

### Recommendations:
- Investors are recommended to capitalize on these findings and open new restaurant in neighborhoods in cluster 0 (shown in purple color) with little competition. 
- Investors with unique cuisine propositions to stand out from the competition can also open new restaurant in neighborhoods in cluster 1 (shown in red color) with moderate competition. 
- Lastly, investors are advised to avoid neighborhoods in cluster 2 (shown in mint green color) which already have high concentration of restaurants and suffering from intense competition.